<a href="https://colab.research.google.com/github/ialara/actf/blob/main/absorption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# F-16 Absorption Model
## Ignacio Lara

### Steps
1. Initialization
  1. Populate the squadron
    * Size (API-1 and API-6)
    * Distribution of INX/EXP
    * Distribution of sortie counts/qualifications/placement in upgrades
1. Inflow new pilots
  * Distribution of B-Course vs. FAIP/PQP (including qualifications of nth-tour)
  * Distribution of API-1 vs API-6
1. Determine resources available
   * Flying Hours remaining
   * Sorties (capacity, driven by UTE and PAA)
   * IPs and/or appropriate upgrade support pilots
1. Determine sortie needs
   * Upgrade vs. CT
   * ASD for FHP accrual
1. Schedule resources for sorties
1. Attempt to fly sorties
   * Refly rate to capture ground abort, incomplete sorties, wx cancellations, etc.
   * Decrement FHP for sorties flown
1. Credit sortie completion for syllabus and/or RTM "beans"
1. Award upgrades / experiencing
1. Outflow pilots

<hr />

### 0. Setup

In [36]:
import numpy as np

rng = np.random.default_rng()

ug_names = ['MQT', 'FLUG', 'IPUG']
ug_rides = [9,     9,      9]
### Simulator rides
# ug_sims =  [7, 5, 6]
###
ug_quals = ['WG',   'FL',   'IP']

class Pilot:
    """An Air Force F-16 pilot that fills squadron billets"""
    # Syllabus lengths (number of UP sorties)
    syllabi_rides = {ug: dur for ug, dur in zip(ug_names, ug_rides)}

    # Quals awarded when completing upgrades
    ug_awards = {ug: q for ug, q in zip(ug_names, ug_quals)}

    # Everyone gets these many sorties from FTU
    ftu_sorties = 59 # Avg based on 49 WG PA release: https://bit.ly/3R8aADh

    # Definition of experience
    exp_sorties = 250
    exp_qual = 'FL'

    def __init__(self, id, f16_sorties, tos, api_category, quals=[], ug=None):
        self.id = id
        self.f16_sorties = self.ftu_sorties + f16_sorties
        self.tos = tos
        self.quals = quals
        self.is_exp = False
        self.check_experience()
        self.api_category = 1 if not self.is_exp else api_category
        self.ug = None
        if ug is not None:
            self.enroll_upgrade(ug)

    def log(self, msg, prefix='>>'):
        print(f'{prefix} PID {self.id} {msg}')

    def enroll_upgrade(self, ug):
        assert self.ug is None, 'Pilot already enrolled in upgrade'
        assert ug in self.ug_awards, 'Invalid upgrade specified'
        assert self.ug_awards[ug] not in self.quals, 'Pilot already completed upgrade'
        self.ug = ug
        self.ride_num = 0
        self.pending_qual = self.ug_awards[self.ug]
        self.log(f'enrolled in {self.ug}')

    def disenroll_upgrade(self):
        prev_ug = self.ug
        self.ug = None
        self.log(f'disenrolled from: {prev_ug}', prefix='<<')

    def fly_ug_sortie(self):
        assert self.ug is not None, 'Pilot not enrolled in upgrade'
        self.ride_num += 1
        if self.check_ug_complete(self.ug, self.ride_num):
          self.award_qual(self.pending_qual)
          self.disenroll_upgrade()
        self.fly_sortie()

    def check_ug_complete(self, ug, ride):
        return ride == self.syllabi_rides[ug]

    def fly_sortie(self):
        self.f16_sorties += 1
        self.check_experience()

    def award_qual(self, qual):
        assert qual in self.ug_awards.values(), 'Invalid qualification specified'
        assert qual not in self.quals, 'Pilot already qualified'
        self.quals.append(qual)
        self.log(f'awarded: {qual}', prefix='++')
        self.check_experience()

    def remove_qual(self, qual):
        assert qual in self.quals, 'Pilot does not have this qualification'
        self.quals.remove(qual)
        self.log(f'un-awarded: {qual}', prefix='--')
        self.check_experience()

    def check_experience(self):
        prev_status = self.is_exp
        self.is_exp = (self.f16_sorties >= self.exp_sorties and 
                      self.exp_qual in self.quals)
        if self.is_exp and not prev_status:
            self.log('EXPERIENCED', prefix='**')

    def increment_tos(self, months=1):
      assert isinstance(months, (int, float)), 'Months must be numeric'
      self.tos += months

    def return_experience(self):
      return self.is_exp

    def return_upgrade(self):
      return self.ug

    def return_api(self):
      return self.api_category

    def return_highest_qual(self):
      if self.quals == []:
        return ''
  
      return self.quals[-1]

    def summarize(self):
        text = 'PID {:2d}: {}{} | SOR: {:4d} | TOS: {:04.1f} mo. | QL: {}'.format(self.id,
                                                            '6-' if self.api_category == 6 else '',
                                                            'EXP' if self.is_exp else 'INX',
                                                            self.f16_sorties,
                                                            self.tos,
                                                            self.quals)
        if self.ug is not None:
            text += f' | UG: {self.ug} #{self.ride_num + 1}' # ride_num + 1 to show next (pending) ride
            if self.ride_num == self.syllabi_rides[self.ug] - 1:
              text += ' (CERT)'
        return text

class SquadronRoster:
    """A collection of Air Force pilots in the same squadron"""
    
    def __init__(self, sq_name, pilots=[]):
        self.sq_name = sq_name
        self.pilots = pilots
        self.pid = 0

    def add_INX_pilot(self, f16_sorties=0, tos=0, quals=[], ug=None):
        pilot = Pilot(self.pid, f16_sorties, tos, api_category=1, quals=quals, ug=ug)
        self.pilots.append(pilot)
        self.pid += 1
        return pilot

    def add_EXP_pilot(self, f16_sorties=250, tos=20, api_category=1, quals=ug_quals[:-1], ug=None):
        pilot = Pilot(self.pid, f16_sorties, tos, api_category, quals, ug)
        self.pilots.append(pilot)
        self.pid += 1
        return pilot

    def populate(self, num_API1=26, prop_INX=0.55, prop_IP=0.2, num_API6=10):
        num_INX = int(num_API1*prop_INX)
        num_EXP = num_API1 - num_INX
        ip_billets_remaining = min(int(num_API1*prop_IP), num_EXP)
        staff_billets_remaining = num_API6

        # Parameters
        max_TOS_INX = 24
        min_TOS_EXP = 24
        max_TOS_EXP = 32

        min_f16_sorties_EXP = Pilot.exp_sorties
        max_f16_sorties_EXP = 500

        max_f16_sorties_INX = Pilot.exp_sorties

        staff_billets_remaining = num_API6
        # Add EXP pilots
        for _ in range(num_EXP + num_API6):
            tos = rng.integers(min_TOS_EXP, max_TOS_EXP + 1)
            # Award sorties proportional to TOS
            sorties = int((tos - min_TOS_EXP)/(max_TOS_EXP - min_TOS_EXP) *
                      (max_f16_sorties_EXP - min_f16_sorties_EXP)) + min_f16_sorties_EXP

            quals = ug_quals[:-1] # Assumed that all EXP are at least FL

            if staff_billets_remaining > 0:
                api = 6 # Assign staff first
                staff_billets_remaining -= 1
            else:
                api = 1
                # Assume only API-1 arrive as IP
                if ip_billets_remaining > 0:
                    quals.append(ug_quals[-1])
                    ip_billets_remaining -= 1
              
            self.add_EXP_pilot(sorties, tos, api, quals)

        # Add INX pilots
        for _ in range(num_INX):
            tos = rng.integers(max_TOS_INX + 1)
            sorties = int(tos/max_TOS_INX * max_f16_sorties_INX)

            quals = []

            pilot = self.add_INX_pilot(sorties, tos, quals=[])

            if pilot.tos > 2:
              pilot.award_qual(ug_quals[0])

            else:
              mqt_progress = rng.integers(pilot.syllabi_rides['MQT']) # Intentionally capping starting INX at 1 MQT ride from end
              pilot.enroll_upgrade('MQT')
              for s in range(mqt_progress):
                pilot.fly_ug_sortie()

    def inflow_nth_tour(self, num_API1, prop_API1_IP, prop_WG, num_API6, prop_API6_IP):
      num_IP = int(num_API1*prop_API1_IP)
      num_INX = int(num_API1*prop_WG)
      num_FL = num_API1 - num_IP - num_INX

      min_sorties_INX = int(0.6*Pilot.exp_sorties)
      max_sorties_INX = int(1.1*Pilot.exp_sorties)

      min_sorties_FL = Pilot.exp_sorties
      max_sorties_FL = int(2.5*Pilot.exp_sorties)

      min_sorties_IP = int(1.5*Pilot.exp_sorties)
      max_sorties_IP = 5*Pilot.exp_sorties

      new_nth_pilots = []

      for _ in range(num_INX):
        sorties = rng.integers(min_sorties_INX, max_sorties_INX)
        pilot = self.add_INX_pilot(f16_sorties=sorties, quals=[ug_quals[0]])
        new_nth_pilots.append(pilot)

      for _ in range(num_FL):
        sorties = rng.integers(min_sorties_FL, max_sorties_FL)
        pilot = self.add_EXP_pilot(f16_sorties=sorties, tos=0, quals=ug_quals[:2])
        new_nth_pilots.append(pilot)

      for _ in range(num_API6):
        sorties = rng.integers(min_sorties_FL, max_sorties_IP)
        is_IP = rng.random() < prop_API6_IP
        my_quals = ug_quals[:2]

        if is_IP:
          my_quals += [ug_quals[-1]]
        pilot = self.add_EXP_pilot(f16_sorties=sorties, tos=0, api_category=6, quals=my_quals)
        new_nth_pilots.append(pilot)

      for _ in range(num_IP):
        sorties = rng.integers(min_sorties_IP, max_sorties_IP)
        pilot = self.add_EXP_pilot(f16_sorties=sorties, tos=0, quals=ug_quals)
        new_nth_pilots.append(pilot)

      return new_nth_pilots

    def inflow_first_tour(self, num_API1):
      new_pilots = []

      for _ in range(num_API1):
        ftu_sortie_delta = rng.integers(-5, 11)
        pilot = self.add_INX_pilot(f16_sorties=ftu_sortie_delta, quals=[])
        new_pilots.append(pilot)

      return new_pilots

      
    def summarize(self):
      pilot_roster = {p.id: {'xp': p.return_experience(),
                              'ug': p.return_upgrade(),
                              'api': p.return_api(),
                              'ql': p.return_highest_qual()} for p in self.pilots}
      # experience = [p.return_experience() for p in self.pilots]
      # upgrades = [p.return_upgrade() for p in self.pilots]
      # apis = [p.return_api() for p in self.pilots]
      # highest_quals = [p.return_highest_qual() for p in self.pilots]

      roster_API1 = {k: v for k, v in pilot_roster.items() if v['api'] == 1}

      num_EXP = np.sum([v['xp'] for v in roster_API1.values()])
      num_INX = len(roster_API1) - num_EXP
      exp_pct = num_EXP / (num_EXP + num_INX)
      
      upgrades = [v['ug'] for v in roster_API1.values()]
      highest_quals = [v['ql'] for v in roster_API1.values()]
      num_MQT = upgrades.count(ug_names[0])
      num_FLUG = upgrades.count(ug_names[1])
      num_IPUG = upgrades.count(ug_names[2])
      num_WG = highest_quals.count(ug_quals[0])
      num_FL = highest_quals.count(ug_quals[1])
      num_IP = highest_quals.count(ug_quals[2])

      print(f'{self.sq_name} SUMMARY:')
      print(
          f'API-1 ({len(roster_API1)}) >> '
          f'EXP: {num_EXP} / INX: {num_INX} | EXP% (API-1): {exp_pct:.2f} | '
          f'API-6: {len(pilot_roster)-len(roster_API1)} | '
          f'{ug_quals[0]}: {num_WG} (+{num_MQT} in {ug_names[0]}) / '
          f'{ug_quals[1]}: {num_FL} (+{num_FLUG} in {ug_names[1]}) / '
          f'{ug_quals[2]}: {num_IP} (+{num_IPUG} in {ug_names[2]}) ')

    
    def print_sq(self):
        print(f'{self.sq_name} ROSTER:')
        for p in self.pilots:
          print(p.summarize())

class F16:
  """A single F-16 jet"""
  hours_between_depot = 500
  depot_days = 90
  mc_rate = 0.7

  def __init__(self, id, num_seats=1, ttaf=2000,
               hours_since_last_depot=500):
    self.id = id
    self.num_seats = num_seats
    self.mc_random_draw()
    self.ttaf = ttaf
    self.hours_since_last_depot = hours_since_last_depot

  def log(self, msg, prefix='>>'):
    print(f'{prefix} {self.id} {msg}')

  def set_nmc(self):
    self.log('is NMC', prefix='<<')
    self.is_mc = False

  def set_mc(self):
    self.log('is MC')
    self.is_mc = True

  def mc_random_draw(self):
    self.is_mc = rng.random() <= self.mc_rate

  def fly_hours(self, hours):
    assert self.is_mc, "Cannot fly NMC aircraft"
    self.ttaf += hours
    self.hours_since_last_depot += hours
    self.log(f'flew {hours} hours. TTAF: {self.ttaf}. Hrs since depot: {self.hours_since_last_depot}', prefix='++')
    if self.needs_depot():
      self.start_depot()

  def sortie(self, duration):
    if self.is_mc:
      self.fly_hours(duration)
    else:
      self.log('not flown due to NMC.', prefix='<X>')

  def needs_depot(self):
    return self.hours_since_last_depot >= self.hours_between_depot

  def start_depot(self):
    self.log('sent to depot!', prefix='!!')
    self.set_nmc()
    self.days_left_in_depot = self.depot_days

  def sit_in_depot(self):
    assert self.days_left_in_depot > 0, "Aircraft already overdue from depot"
    self.days_left_in_depot -= 1

  def complete_depot(self):
    self.log('depot complete.', prefix='**')
    self.set_mc()
    self.days_left_in_depot = -1
    self.hours_since_last_depot = 0

  def print_ac(self):
    print(f'{self.id} | MC?: {self.is_mc} | TTAF: {self.ttaf} | Hrs. since depot: {self.hours_since_last_depot}')

class SquadronHangar:
  """A collection of F-16s assigned to a squadron"""
  def __init__(self, jets=[]):
    pass

  def populate(self):
    pass

  def summarize(self):
    pass

  def make_fly_order(self):
    pass


class DailySchedule:
  """Pairing of crews and aircraft """
  pass






## Pilot class example

Contains methods and properties associated with individual pilots. For example:

_Methods_
- Fly a sortie
- Enroll in/disenroll from upgrade
- Become experienced

_Properties_
- \# sorties
- Time on Station
- API category
- Inexperienced/Experienced status
- Qualifications
- Upgrade progress

Class methods also contain safeguards against improper usage. In this example, we first create an `example_pilot`, with `pid = 0` (pilot ID), `f16_sorties = 24`, `tos = 4` months, and an `API=1`, with no qualifications. Then, we:
1. Fly a single sortie for the pilot, updating total sortie count only.
1. Attempt to fly an upgrade sortie, even though the pilot is not enrolled in an upgrade (will raise an `AssertionError`). 

In [2]:
example_pilot = Pilot(0, 24, 4, 1, [])
print(example_pilot.summarize())

example_pilot.fly_sortie()
print('---After flying sortie---')
print(example_pilot.summarize())
print('---Attempt to fly upgrade sortie with no upgrade---')
example_pilot.fly_ug_sortie()

PID  0: INX | API: 1 | SOR:   39 | TOS: 04.0 mo. | QL: []
---After flying sortie---
PID  0: INX | API: 1 | SOR:   40 | TOS: 04.0 mo. | QL: []
---Attempt to fly upgrade sortie with no upgrade---


AssertionError: ignored

We fix the `AssertionError` by enrolling the pilot in MQT. Now, it makes sense to fly an upgrade sortie, which updates totals like before, as well as progressing the pilot through an upgrade syllabus and checking for completion.
1. The pilot's progress through MQT is indicated by `UG: MQT #0` (only when enrolled in an upgrade). When the pilot completes MQT, the pilot is awarded the Wingman qualification, shown in `QL = ['WG']`, and disenrolled from MQT.
1. When the pilot meets 250 sorties, but without completing FLUG, the pilot is still recognized as inexperienced (`INX`).
1. After the pilot completes FLUG, and meets the sortie minimum, the pilot is awarded the FL qualification, disenrolled from FLUG, and updated to reflect experienced status (`EXP`).

In practice, these methods and properties will be called by higher-level functions in the simulation. But this example demonstrates some of the functionality present in the `Pilot` class.

In [3]:
example_pilot.enroll_upgrade('MQT')
print(example_pilot.summarize())

example_pilot.fly_ug_sortie()
print('---After flying MQT sortie---')
print(example_pilot.summarize())

>> PID 0 enrolled in MQT
PID  0: INX | API: 1 | SOR:   40 | TOS: 04.0 mo. | QL: [] | UG: MQT #1
---After flying MQT sortie---
PID  0: INX | API: 1 | SOR:   41 | TOS: 04.0 mo. | QL: [] | UG: MQT #2


In [4]:
print('---Fly remaining MQT sorties---')
for _ in range(example_pilot.syllabi_rides[example_pilot.ug] - example_pilot.ride_num):
  example_pilot.fly_ug_sortie()
print(example_pilot.summarize())
print()

example_pilot.f16_sorties = 250
print('---Meet EXP sortie criteria, but not FL---')
print(example_pilot.summarize())
print()

example_pilot.enroll_upgrade('FLUG')

for _ in range(example_pilot.syllabi_rides[example_pilot.ug] - 1 - example_pilot.ride_num):
  example_pilot.fly_ug_sortie()

print('---One ride from completing FLUG---')
print(example_pilot.summarize())
print()

example_pilot.fly_ug_sortie()
print('---After completing FLUG, now meets criteria for EXP---')
print(example_pilot.summarize())


---Fly remaining MQT sorties---
++ PID 0 awarded: WG
<< PID 0 disenrolled from: MQT
PID  0: INX | API: 1 | SOR:   49 | TOS: 04.0 mo. | QL: ['WG']

---Meet EXP sortie criteria, but not FL---
PID  0: INX | API: 1 | SOR:  250 | TOS: 04.0 mo. | QL: ['WG']

>> PID 0 enrolled in FLUG
---One ride from completing FLUG---
PID  0: INX | API: 1 | SOR:  258 | TOS: 04.0 mo. | QL: ['WG'] | UG: FLUG #9 (CERT)

++ PID 0 awarded: FL
** PID 0 EXPERIENCED
<< PID 0 disenrolled from: FLUG
---After completing FLUG, now meets criteria for EXP---
PID  0: EXP | API: 1 | SOR:  259 | TOS: 04.0 mo. | QL: ['WG', 'FL']


In [5]:
example_pilot.increment_tos('three')

AssertionError: ignored

In [6]:
example_pilot.increment_tos()
print(example_pilot.summarize())

example_pilot.increment_tos(1.5)
print(example_pilot.summarize())

example_pilot.increment_tos(-2.5)
print(example_pilot.summarize())

PID  0: EXP | API: 1 | SOR:  259 | TOS: 05.0 mo. | QL: ['WG', 'FL']
PID  0: EXP | API: 1 | SOR:  259 | TOS: 06.5 mo. | QL: ['WG', 'FL']
PID  0: EXP | API: 1 | SOR:  259 | TOS: 04.0 mo. | QL: ['WG', 'FL']


In [7]:
example_pilot.remove_qual('FL')
print(example_pilot.summarize())

example_pilot.award_qual('FL')
print(example_pilot.summarize())

-- PID 0 un-awarded: FL
PID  0: INX | API: 1 | SOR:  259 | TOS: 04.0 mo. | QL: ['WG']
++ PID 0 awarded: FL
** PID 0 EXPERIENCED
PID  0: EXP | API: 1 | SOR:  259 | TOS: 04.0 mo. | QL: ['WG', 'FL']


## SquadronRoster example

Contains the Pilots (API-1 and API-6) present in a single F-16 squadron. Has some methods and properties that make sense at the squadron level. For example:

_Methods_:
- Add/remove individual Pilots
- Populate an initial squadron with desired size, EXP%, and qualifications

_Properties_:
- Squadron name
- Squadron summary
- Summary statistics (EXP%, upgrade sizes, etc.)

In this example, we create a notional squadron.

The defaults for `SquadronRoster.populate` include:
- `size=10` pilots
- `prop_INX=0.55` percent inexperienced pilots
- `prop_IP=0.3` percent _of experienced API-1 pilots_ qualified as instructors
- `num_API6=2` billets for API-6 staff

In [37]:
sq = SquadronRoster('0th FS', pilots=[])
sq.populate()
sq.print_sq()

** PID 0 EXPERIENCED
** PID 1 EXPERIENCED
** PID 2 EXPERIENCED
** PID 3 EXPERIENCED
** PID 4 EXPERIENCED
** PID 5 EXPERIENCED
** PID 6 EXPERIENCED
** PID 7 EXPERIENCED
** PID 8 EXPERIENCED
** PID 9 EXPERIENCED
** PID 10 EXPERIENCED
** PID 11 EXPERIENCED
** PID 12 EXPERIENCED
** PID 13 EXPERIENCED
** PID 14 EXPERIENCED
** PID 15 EXPERIENCED
** PID 16 EXPERIENCED
** PID 17 EXPERIENCED
** PID 18 EXPERIENCED
** PID 19 EXPERIENCED
** PID 20 EXPERIENCED
** PID 21 EXPERIENCED
++ PID 22 awarded: WG
>> PID 23 enrolled in MQT
++ PID 24 awarded: WG
++ PID 25 awarded: WG
++ PID 26 awarded: WG
++ PID 27 awarded: WG
++ PID 28 awarded: WG
++ PID 29 awarded: WG
++ PID 30 awarded: WG
++ PID 31 awarded: WG
++ PID 32 awarded: WG
++ PID 33 awarded: WG
++ PID 34 awarded: WG
++ PID 35 awarded: WG
0th FS ROSTER:
PID  0: 6-EXP | SOR:  340 | TOS: 25.0 mo. | QL: ['WG', 'FL']
PID  1: 6-EXP | SOR:  559 | TOS: 32.0 mo. | QL: ['WG', 'FL']
PID  2: 6-EXP | SOR:  496 | TOS: 30.0 mo. | QL: ['WG', 'FL']
PID  3: 6-EXP | 

In [38]:
sq.summarize()

0th FS SUMMARY:
API-1 (26) >> EXP: 12 / INX: 14 | EXP% (API-1): 0.46 | API-6: 10 | WG: 13 (+1 in MQT) / FL: 7 (+0 in FLUG) / IP: 5 (+0 in IPUG) 


## 1. Inflow Pilots

In [39]:
new_pilots = sq.inflow_nth_tour(num_API1=10, prop_API1_IP=.4, prop_WG=.2, 
                                num_API6=5, prop_API6_IP=.6)

sq.summarize()


** PID 38 EXPERIENCED
** PID 39 EXPERIENCED
** PID 40 EXPERIENCED
** PID 41 EXPERIENCED
** PID 42 EXPERIENCED
** PID 43 EXPERIENCED
** PID 44 EXPERIENCED
** PID 45 EXPERIENCED
** PID 46 EXPERIENCED
** PID 47 EXPERIENCED
** PID 48 EXPERIENCED
** PID 49 EXPERIENCED
** PID 50 EXPERIENCED
0th FS SUMMARY:
API-1 (36) >> EXP: 20 / INX: 16 | EXP% (API-1): 0.56 | API-6: 15 | WG: 15 (+1 in MQT) / FL: 11 (+0 in FLUG) / IP: 9 (+0 in IPUG) 


In [40]:
[p.summarize() for p in new_pilots]

["PID 36: INX | SOR:  252 | TOS: 00.0 mo. | QL: ['WG']",
 "PID 37: INX | SOR:  211 | TOS: 00.0 mo. | QL: ['WG']",
 "PID 38: EXP | SOR:  473 | TOS: 00.0 mo. | QL: ['WG', 'FL']",
 "PID 39: EXP | SOR:  538 | TOS: 00.0 mo. | QL: ['WG', 'FL']",
 "PID 40: EXP | SOR:  342 | TOS: 00.0 mo. | QL: ['WG', 'FL']",
 "PID 41: EXP | SOR:  339 | TOS: 00.0 mo. | QL: ['WG', 'FL']",
 "PID 42: 6-EXP | SOR:  996 | TOS: 00.0 mo. | QL: ['WG', 'FL', 'IP']",
 "PID 43: 6-EXP | SOR: 1095 | TOS: 00.0 mo. | QL: ['WG', 'FL']",
 "PID 44: 6-EXP | SOR:  979 | TOS: 00.0 mo. | QL: ['WG', 'FL', 'IP']",
 "PID 45: 6-EXP | SOR: 1022 | TOS: 00.0 mo. | QL: ['WG', 'FL', 'IP']",
 "PID 46: 6-EXP | SOR:  940 | TOS: 00.0 mo. | QL: ['WG', 'FL']",
 "PID 47: EXP | SOR: 1215 | TOS: 00.0 mo. | QL: ['WG', 'FL', 'IP']",
 "PID 48: EXP | SOR: 1288 | TOS: 00.0 mo. | QL: ['WG', 'FL', 'IP']",
 "PID 49: EXP | SOR:  791 | TOS: 00.0 mo. | QL: ['WG', 'FL', 'IP']",
 "PID 50: EXP | SOR:  849 | TOS: 00.0 mo. | QL: ['WG', 'FL', 'IP']"]

In [41]:
new_pilots = sq.inflow_first_tour(8)
sq.summarize()

0th FS SUMMARY:
API-1 (44) >> EXP: 20 / INX: 24 | EXP% (API-1): 0.45 | API-6: 15 | WG: 15 (+1 in MQT) / FL: 11 (+0 in FLUG) / IP: 9 (+0 in IPUG) 


In [42]:
[p.summarize() for p in new_pilots]

['PID 51: INX | SOR:   62 | TOS: 00.0 mo. | QL: []',
 'PID 52: INX | SOR:   55 | TOS: 00.0 mo. | QL: []',
 'PID 53: INX | SOR:   67 | TOS: 00.0 mo. | QL: []',
 'PID 54: INX | SOR:   60 | TOS: 00.0 mo. | QL: []',
 'PID 55: INX | SOR:   65 | TOS: 00.0 mo. | QL: []',
 'PID 56: INX | SOR:   55 | TOS: 00.0 mo. | QL: []',
 'PID 57: INX | SOR:   59 | TOS: 00.0 mo. | QL: []',
 'PID 58: INX | SOR:   61 | TOS: 00.0 mo. | QL: []']